In [5]:
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 3.2 MB/s eta 0:00:00


In [3]:
!pip3 install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 16.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 16.7 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system =

In [3]:

from datasets import load_dataset

ds = load_dataset("rajpurkar/squad_v2")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/8.92k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/16.4M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/1.35M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/130319 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/11873 [00:00<?, ? examples/s]

In [5]:
ds

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 130319
    })
    validation: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 11873
    })
})

In [6]:
ds['train'].to_csv('train.csv')

Creating CSV from Arrow format:   0%|          | 0/131 [00:00<?, ?ba/s]

124251882

In [7]:
ds['validation'].to_csv('validation.csv')

Creating CSV from Arrow format:   0%|          | 0/12 [00:00<?, ?ba/s]

12397528

In [1]:
from datasets import load_dataset
from transformers import AutoTokenizer
import torch

model_checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

squad = load_dataset("squad_v2")

# Select a smaller subset
small_train_dataset = squad["train"].select(range(2000))      # ~2K samples
small_val_dataset = squad["validation"].select(range(500))    # ~500 samples

def prepare_train_features(example):
    # Tokenize question and context together
    tokenized = tokenizer(
        example["question"],
        example["context"],
        truncation="only_second",  # Only truncate context
        max_length=384,
        stride=128,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length"
    )

    sample_mapping = tokenized.pop("overflow_to_sample_mapping")
    offset_mapping = tokenized.pop("offset_mapping")

    start_positions = []
    end_positions = []

    for i, offsets in enumerate(offset_mapping):
        input_ids = tokenized["input_ids"][i]
        cls_index = input_ids.index(tokenizer.cls_token_id)
        sequence_ids = tokenized.sequence_ids(i)

        sample_idx = sample_mapping[i]
        answers = example["answers"][sample_idx]

        if len(answers["answer_start"]) == 0:
            start_positions.append(cls_index)
            end_positions.append(cls_index)
        else:
            start_char = answers["answer_start"][0]
            end_char = start_char + len(answers["text"][0])
            context_start = sequence_ids.index(1)
            context_end = len(sequence_ids) - 1 - sequence_ids[::-1].index(1)

            token_start_index = context_start
            token_end_index = context_end

            while token_start_index < len(offsets) and offsets[token_start_index][0] <= start_char:
                token_start_index += 1
            start_positions.append(token_start_index - 1)

            while token_end_index >= 0 and offsets[token_end_index][1] >= end_char:
                token_end_index -= 1
            end_positions.append(token_end_index + 1)

    tokenized["start_positions"] = start_positions
    tokenized["end_positions"] = end_positions
    return tokenized

tokenized_train = small_train_dataset.map(prepare_train_features, batched=True, remove_columns=squad["train"].column_names)
tokenized_val = small_val_dataset.map(prepare_train_features, batched=True, remove_columns=squad["validation"].column_names)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

In [2]:
from transformers import AutoModelForQuestionAnswering, TrainingArguments, Trainer
import os
os.environ["WANDB_DISABLED"] = "true"
model = AutoModelForQuestionAnswering.from_pretrained(model_checkpoint)

training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=2,
    weight_decay=0.01,
    logging_dir='./logs',
    save_total_limit=2
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_val,
    tokenizer=tokenizer
)

trainer.train()


Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1611: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
<ipython-input-2-33c43e018e82>:18: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss
1,No log,5.075490
2,2.569700,5.460599


TrainOutput(global_step=514, training_loss=2.546972612462619, metrics={'train_runtime': 332.8534, 'train_samples_per_second': 12.318, 'train_steps_per_second': 1.544, 'total_flos': 803487526963200.0, 'train_loss': 2.546972612462619, 'epoch': 2.0})

In [7]:
model.push_to_hub("piyushjoshi/qa-custom-model", private=True)
tokenizer.push_to_hub("piyushjoshi/qa-custom-model", private=True)

model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/piyushjoshi/qa-custom-model/commit/5c731742ab80689457975aa51fa5e96f1b5c1cde', commit_message='Upload tokenizer', commit_description='', oid='5c731742ab80689457975aa51fa5e96f1b5c1cde', pr_url=None, repo_url=RepoUrl('https://huggingface.co/piyushjoshi/qa-custom-model', endpoint='https://huggingface.co', repo_type='model', repo_id='piyushjoshi/qa-custom-model'), pr_revision=None, pr_num=None)

Now evaluate the model

In [7]:
from transformers import pipeline, AutoTokenizer, AutoModelForQuestionAnswering
from datasets import load_dataset
import evaluate
import torch

# Load fine-tuned model and tokenizer
model = AutoModelForQuestionAnswering.from_pretrained("piyushjoshi/qa-custom-model")  # Replace with your uploaded model
tokenizer = AutoTokenizer.from_pretrained("piyushjoshi/qa-custom-model")

# Set device
device = 0 if torch.cuda.is_available() else -1
print("Using CUDA" if device == 0 else "Using CPU")

# Load evaluation dataset
dataset = load_dataset("squad_v2", split="validation[:100]")  # Small subset for quick testing

# Load QA pipeline
qa_pipeline = pipeline("question-answering", model=model, tokenizer=tokenizer, device=device)

# Evaluation metric
metric = evaluate.load("squad_v2")

# Apply pipeline in batches using map
def get_prediction(example):
    result = qa_pipeline(question=example["question"], context=example["context"])
    return {
        "id": example["id"],
        "prediction_text": result["answer"],
        "no_answer_probability": 0.0  # Required for squad_v2 metric
    }

predictions_dataset = dataset.map(get_prediction)

# Prepare lists for metric computation
predictions = [{
    "id": pred["id"],
    "prediction_text": pred["prediction_text"],
    "no_answer_probability": pred["no_answer_probability"]
} for pred in predictions_dataset]

references = [{
    "id": example["id"],
    "answers": example["answers"]
} for example in dataset]

# Compute metrics
results = metric.compute(predictions=predictions, references=references)
print("📊 Evaluation Results:", results)


Using CUDA


Device set to use cuda:0


Map:   0%|          | 0/100 [00:00<?, ? examples/s]

📊 Evaluation Results: {'exact': 26.0, 'f1': 30.265873015873016, 'total': 100, 'HasAns_exact': 57.77777777777778, 'HasAns_f1': 67.25749559082892, 'HasAns_total': 45, 'NoAns_exact': 0.0, 'NoAns_f1': 0.0, 'NoAns_total': 55, 'best_exact': 58.0, 'best_exact_thresh': 0.0, 'best_f1': 58.0, 'best_f1_thresh': 0.0}
